# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 ноября 2017, 23:59   
**Штраф за опоздание:** -2 балла после 23:59  4 декабря, -4 балла после 23:59 11 декабря, -6 баллов после 23:59 18 декабря

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

**Разбаловка:**   
За задание можно получить 10 баллов. Для этого нужно следующее:
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает    
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала качественнее, чем у дерева из sklearn

Лучшее разбиение $$ Gain(S, A) = I(S) - \left(\frac{|S_L|}{|S|}\cdot I(S_L) + \frac{|S_R|}{|S|}\cdot I(S_R) \right),$$

Gini index $$I(S) = 1 - \sum\limits_k (p_k)^2 = \sum\limits_{k'\neq k} p_{k'} p_k$$
Entropy $$I(S) = -\sum\limits_k p_k \log(p_k)$$
Missclassification error
$$I(S) = 1 - \max\limits_k p_k $$

In [52]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [53]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [54]:
%%cython

import numpy as np
cimport numpy as np
import cython

@cython.boundscheck(False)
@cython.cdivision(True)
@cython.wraparound(False)
cpdef void sort_samples(np.ndarray[float, ndim=1] x, int[:] y):
        cdef long[:] idx_sorted = x.argsort()
        cdef float[:] x_copy = x.copy()
        cdef int[:]   y_copy = y.copy()
        for i in range(x.shape[0]):
            x_copy[i] = x[idx_sorted[i]]
            y_copy[i] = y[idx_sorted[i]]

In [55]:
class MineDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        p_l = l_c/l_s.astype('float')
        p_r = r_c/r_s.astype('float')

        gini_left  = 1-np.sum(p_l**2, axis = 1) # left  score
        gini_right = 1-np.sum(p_r**2, axis = 1) # right score

        return gini_left/l_s.ravel() + gini_right/r_s.ravel()
             
    def __entropy(self, l_c, l_s, r_c, r_s): #very very long
        p_l = l_c/l_s.astype('float')+0.00001
        p_r = r_c/r_s.astype('float')+0.00001

        score_left  = 1-np.sum(np.log(p_l)*p_l, axis = 1)  # left  score
        score_right = 1-np.sum(np.log(p_l)*p_l, axis = 1)  # right score
        return score_left/l_s.ravel() + score_right/r_s.ravel()

    def __misclass(self, l_c, l_s, r_c, r_s):
        p_l = l_c/l_s.astype('float')
        p_r = r_c/r_s.astype('float')

        score_left  = 1 - np.max(p_l, axis = 1)
        score_right = 1 - np.max(p_r, axis = 1)
        return score_left/l_s.ravel() + score_right/r_s.ravel()

    def __get_feature_ids_sqrt(self, n_feature):
        return np.random.choice(np.arange(n_feature), size = int(np.sqrt(n_feature)), replace = False)
        
    def __get_feature_ids_log2(self, n_feature):
        return np.random.choice(np.arange(n_feature), size = int(np.log2(n_feature)), replace = False)

    def __get_feature_ids_N(self, n_feature):
        return np.random.choice(np.arange(n_feature), size = n_feature, replace = False)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()#.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Sort x,y by x
        sorted_x = x.copy()
        sorted_y = y.copy()
        sort_samples(x, y)

        # number of classes
        class_number = np.unique(y).shape[0]
        
        #sorted_y[min_samples:-min_samples]
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        #indexes where y changing [1,1,0,0,1,0] -> [2,4,5]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        #if y = const - return +inf
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # number of samples in sequense where y not changed [1,1,0,0,1,0] -> [2,2,1,1]
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        # one hot encoding of y (for y that changed ) [1,1,0,0,1,0] -> [[0,1],[1,0],[0,1],[1,0]]
        one_hot_code = np.zeros((r_border_ids.shape[0], self.num_class))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        
        #num of  one-hot encoding of classes in split [1,1,0,0,1,0] -> [[0,2],[2,0],[0,1],[1,0]]
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(y[:self.min_samples_split], minlength=self.num_class)
        
        #num of  one-hot encoding of classes before that [1,1,0,0,1,0] -> [[0,2],[2,2],[2,3],[3,3]]
        l_class_count = np.cumsum(class_increments, axis=0)
        #num of  one-hot encoding of classes (from left to right) before that [1,1,0,0,1,0] -> [[3,1],[1,1],[1,0],[0,0]]
        r_class_count = np.bincount(y) - l_class_count
        #num of elements left
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        #num of elements right
        r_sizes = sorted_y.shape[0] - l_sizes

        #information criterion os all split
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        #find min
        idx = np.argmin(gs)
    
        #split_value
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0
             
    def find_threshold(self, x, y):
        return self. __find_threshold(x, y)

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        
        if depth < self.max_depth and len(y)>self.min_samples_split: 
            MIN_SPLIT = [np.inf, -1, -1]

            for feature_id in self.get_feature_ids(self.num_features):
                min_gini, value = self.find_threshold(x[:, feature_id], y)
                if min_gini<MIN_SPLIT[0]:
                    MIN_SPLIT = min_gini, feature_id, value
            if MIN_SPLIT[0] == np.inf:
                probs = np.bincount(y, minlength = self.num_class)/len(y)
                self.tree[node_id] = [self.LEAF_TYPE, np.argmax(probs), probs]
            else:
                self.tree[node_id] = [self.NON_LEAF_TYPE, *MIN_SPLIT[1:]]

            x_left, x_right, y_left, y_right = self.__div_samples(x, y, MIN_SPLIT[1], MIN_SPLIT[2])
            if len(y_left) >= self.min_samples_split and len(y_right) >= self.min_samples_split:
                if len(set(y_left)) != 1:
                    self.__fit_node(x_left, y_left, node_id * 2, depth+1)
                else:
                    probs = np.bincount(y_left, minlength = self.num_class)/len(y_left)   
                    self.tree[node_id*2] = [self.LEAF_TYPE, y_left[0], probs]
                    
                if len(set(y_right)) != 1:
                    self.__fit_node(x_right, y_right, node_id * 2+1, depth+1)
                else:
                    probs = np.bincount(y_right, minlength = self.num_class)/len(y_right)   
                    self.tree[node_id*2+1] = [self.LEAF_TYPE, y_right[0], probs]


                return
        probs = np.bincount(y, minlength = self.num_class)/len(y)
        self.tree[node_id] = [self.LEAF_TYPE, np.argmax(probs), probs]
        
        return 
    
    def fit(self, x, y):
        x = x.astype(np.float32)
        y = y.astype(np.int32)
        self.num_class = np.unique(y).size
        self.num_features = x.shape[1]
        self.__fit_node(x, y, 1, 0)
        return self
        
    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id)
            else:
                return self.__predict_class(x, 2 * node_id + 1)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 1) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [56]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [57]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

## Проверка скорости работы

In [60]:
my_clf = MineDecisionTreeClassifier(min_samples_split=2, max_depth = 5, criterion = 'entropy')
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)


clf = DecisionTreeClassifier(min_samples_split=2, max_depth = 5, criterion = 'entropy')
t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.36174893379211426
0.3880009651184082


## Проверка качества работы

In [61]:
gkf = KFold(n_splits=5, shuffle=True)

In [69]:
my_clf = MineDecisionTreeClassifier(min_samples_split=2, max_depth = 5, criterion='gini')
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    res = accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test)
    results['mine'].append(res)
    print(res)

0.932693107176
0.934979629168
0.929533549514
0.932859399684
0.933563380867


In [70]:
clf = DecisionTreeClassifier(min_samples_split=2, max_depth = 5, criterion='gini')
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    res = accuracy_score(y_pred=clf.predict(X_test), y_true=y_test)
    results['sklearn'].append(res)
    print(res)

0.931570632743
0.930240292675
0.929575122641
0.933566142845
0.932191410635


####  